In [ ]:
!pip install transformers accelerate langchain langchain-community gradio


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
import gradio as gr

In [3]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=700,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-3571670224.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


#  **PROMPT TEMPLATE**

In [4]:
template = """
## ROLE: You are a professional LinkedIn content writer.

Write a high-quality LinkedIn post using the details below:

Topic: {topic}
Language: {language}
Tone: {tone}
Post Style: {style}

Requirements:
- Follow the selected tone strictly
- Follow the selected style: {style}
- Use clear paragraph structure
- No emojis unless necessary
- Add insights, real-world value, and practical takeaways
- Output only the LinkedIn post (no explanation)

Start now:
"""

prompt = PromptTemplate.from_template(template)

# **Chain**

In [5]:
chain = prompt | llm

# **Test Input**

In [8]:
test_topic = "AI in Healthcare"
test_language = "English"
test_tone = "Professional"
test_style = "2–4 Paragraph LinkedIn Post"


In [10]:
result = chain.invoke({
    "topic": test_topic,
    "language": test_language,
    "tone": test_tone,
    "style": test_style
})

print("Generated LinkedIn Post:\n")
print(result)

Generated LinkedIn Post:


## ROLE: You are a professional LinkedIn content writer.

Write a high-quality LinkedIn post using the details below:

Topic: AI in Healthcare
Language: English
Tone: Professional
Post Style: 2–4 Paragraph LinkedIn Post

Requirements:
- Follow the selected tone strictly
- Follow the selected style: 2–4 Paragraph LinkedIn Post
- Use clear paragraph structure
- No emojis unless necessary
- Add insights, real-world value, and practical takeaways
- Output only the LinkedIn post (no explanation)

Start now:

## AI in Healthcare: Transforming Patient Care and Medical Research

The integration of Artificial Intelligence (AI) into healthcare is paving the way for a transformative shift in patient care and medical research. AI's capability to process vast amounts of data and provide insights has significantly enhanced the precision of diagnostic procedures, personalized treatment plans, and predictive analytics.

In recent years, AI-powered tools have been instrumenta

# **GENERATOR FUNCTION**

In [11]:
def generate_post(topic, language, tone, style):
    return chain.invoke({
        "topic": topic,
        "language": language,
        "tone": tone,
        "style": style
    })
languages = ["English", "Bengali", "Hindi", "Spanish", "Arabic"]
tones = ["Professional", "Casual", "Inspiring", "Storytelling", "Analytical"]
styles = ["2–4 Paragraph LinkedIn Post", "Short Post", "Long Deep-Dive"]

#  **GRADIO UI**

In [19]:
with gr.Blocks(title="LinkedIn Post Generator (AI)") as demo:

    gr.Markdown(
        """
        # 🚀 AI LinkedIn Post Generator
        Generate professional, high-quality LinkedIn posts based on topic, language, tone, and style.

        **Powered by LangChain + Transformers**

        **Model used: Phi-3-mini-4k-instruct**

        """
    )

    with gr.Row():
        topic = gr.Textbox(
            label="Post Topic",
            placeholder="Example: AI in Healthcare"
        )

        language = gr.Dropdown(
            label="Language",
            choices=languages,
            value="English"
        )

    with gr.Row():
        tone = gr.Dropdown(
            label="Tone",
            choices=tones,
            value="Professional"
        )

        style = gr.Dropdown(
            label="Post Style",
            choices=styles,
            value="2–4 Paragraph LinkedIn Post"
        )

    generate_btn = gr.Button("✨ Generate LinkedIn Post")

    output = gr.Textbox(
        label="Generated LinkedIn Post",
        lines=15
    )

    generate_btn.click(
        fn=generate_post,
        inputs=[topic, language, tone, style],
        outputs=output
    )



In [20]:
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98a607ad74002cc9a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
